In [91]:
#Importaing necessary libraries
import pandas as pd

In [92]:
#Loading the datasets
df1 = pd.read_csv('takehome_users.csv')
df2 = pd.read_csv('takehome_user_engagement.csv')

In [93]:
#Getting an idea of the first dataset
df1.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [94]:
#Finding out the number of rows and columns the dataset consists of
df1.shape

(12000, 10)

In [95]:
#Finding out the properties of the first dataset
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [96]:
#Dropped columns with missing values that will make the data easier to work with; these 2 columns aren't that useful to 
#address our inquiry anyway.
df1 = df1.drop(columns=['last_session_creation_time', 'invited_by_user_id'])

In [97]:
#Info of the revised dataset
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   object_id                   12000 non-null  int64 
 1   creation_time               12000 non-null  object
 2   name                        12000 non-null  object
 3   email                       12000 non-null  object
 4   creation_source             12000 non-null  object
 5   opted_in_to_mailing_list    12000 non-null  int64 
 6   enabled_for_marketing_drip  12000 non-null  int64 
 7   org_id                      12000 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 750.1+ KB


In [98]:
#Further revising the original dataset as I guage these features don't play too much a critical role of pivotal features
#in determining 'adopted users'.
df1 = df1.drop(columns=['creation_time', 'name', 'email',])

In [99]:
#Finding out properties of our newly revised dataset
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   object_id                   12000 non-null  int64 
 1   creation_source             12000 non-null  object
 2   opted_in_to_mailing_list    12000 non-null  int64 
 3   enabled_for_marketing_drip  12000 non-null  int64 
 4   org_id                      12000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 468.9+ KB


In [69]:
#Getting an idea of the second dataset
df2.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [70]:
#Finding out the size of the dataset; Rows:Columns
df2.shape

(207917, 3)

In [71]:
#Getting a count of the number of unique values in this column; this column will play a pivotal role in our inquiry
df2['user_id'].value_counts()

3623     606
906      600
1811     593
7590     590
8068     585
        ... 
3267       1
1218       1
7361       1
11335      1
2047       1
Name: user_id, Length: 8823, dtype: int64

In [72]:
#Reading the second csv file into a dataframe again to parse the dates by weeks
df2 = pd.read_csv('takehome_user_engagement.csv', parse_dates=['time_stamp'])

In [73]:
#Creating new column where timestamps are parsed by week
df2['weeks'] = df2['time_stamp'].dt.isocalendar().week

In [74]:
#Grouping by 'weeks' and 'user_id' and resetting the index
df2 = df2.groupby(['weeks', 'user_id'])['time_stamp'].count().reset_index(name='amount')

In [75]:
#Preparing a list of user_ids that meet this requirement of logging in at least 3 times in a one week interval
result = df2[df2['amount'] >= 3 ]

In [77]:
#Creating a list of user_ids that meet the requirements of our inquiry
user_ids = result.user_id.tolist()
#print(user_ids)

In [79]:
#Getting the amount of user_ids that meet the criteria by using len function
print(len(user_ids))

29771


In [81]:
#Creating a list of tuples consisting of user_ids and amount of times logged-in
user_amount = [(x, y) for x, y in zip(result.user_id, result.amount)]
#print(user_amount)

In [114]:
#Creating a new list to use as a delineater for 'adopted users' and 'non adopted users'
new_column = [user for user in df1['object_id'] if user in result['user_id']]

In [121]:
#How the dataframe looks now
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   object_id                   12000 non-null  int64 
 1   creation_source             12000 non-null  object
 2   opted_in_to_mailing_list    12000 non-null  int64 
 3   enabled_for_marketing_drip  12000 non-null  int64 
 4   org_id                      12000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 468.9+ KB


In [122]:
#Creating a new column for the first dataframe that delineates between adopted user and non adopted user to use as a 'label'
#column
df1['user_id'] = [1 if user in new_column else 0 for user in df1['object_id']]

In [127]:
#Replacing the old values of 'creation_source' column with integers so it will work with the algorithm to be used
df1['creation_source'] = df1['creation_source'].replace(['GUEST_INVITE', 'ORG_INVITE', 'SIGNUP', 'PERSONAL_PROJECTS',
                                                        'SIGNUP_GOOGLE_AUTH'], [1, 2, 3, 4, 5])

In [130]:
#Dropping the old column 'object_id' as it has been replaced with 'user_id' which has labelled the values in boolean-like
#form to distinguish between adopted users and non users.
df1 = df1.drop(columns=['object_id'])

In [131]:
#Now we have the dataframe we want to work with to discover how much of a factor these features are in predicting user
#adoption
df1.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,user_id
0,1,1,0,11,1
1,2,0,0,1,1
2,2,0,0,94,0
3,1,0,0,1,0
4,1,0,0,193,1


In [132]:
#Separating the data into labels and attributes to figure out how much these features influence the labels of adopted user
#and non adopted user
X = df1.drop('user_id', axis=1)
y = df1['user_id']

In [150]:
#Dividing up the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [151]:
#Creating an SVC object and training the algorithm on the training data
from sklearn.svm import SVC
svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(X_train, y_train);

In [152]:
#Making a prediction on the test set
y_pred = svclassifier.predict(X_test)

In [153]:
#Evaluating the algorithm with a confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[491 585]
 [604 720]]
              precision    recall  f1-score   support

           0       0.45      0.46      0.45      1076
           1       0.55      0.54      0.55      1324

    accuracy                           0.50      2400
   macro avg       0.50      0.50      0.50      2400
weighted avg       0.51      0.50      0.50      2400



In [ ]:
#As can be seen in the confusion matrix above,the SVM classifier predicted just below a 50% accuracy level for non-adopted
#user and the classifier predicted a few percentage above the 50% mark for correctly identifying adopted users. The overall
#accuracy for the predictions of the test sets rest at 52%. I tried to run the classifier again but the accuracy acutally
#decreased a bit. I chose the Support Vector Machine algorithm to address the inquiry here as I believed its ability to 
#recognize patterns would be well-suited for our inquiry here in identifying features that factor into customers becoming
#adopted users or not. Clearly, the SVM classifier predicted that the features either did not factor into that trend of 
#adopted users or the current dataset was not best handled by the SVM algorithm. Perhaps an unsupervised machine learning
#algorithm like Naive Bayes classifier would have worked well here and we could have kept the observations with missing
#values as well in that case.In this instance, either the SVM algorithm didn't pan out for our particular inquiry or the
#the selected features are poor indicators of future adopted use; of course, there could be other reasons for the poor
#performance here but those 2 aforementioned reasons immediately come to mind.